In [21]:
import sys

In [22]:
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-0zpcunv6
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-0zpcunv6
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done


In [23]:
from pyknow import *

YES = 'Да'
NO = 'Нет'

ACTION_TURN_AROUND = 'Развернуться'
ACTION_CONTINUE_DRIVING_STRAIGHT = 'Продолжать движение вперёд по правилам'
ACTION_CONTINUE_DRIVING_CROSSROAD = 'Следуем правилам проезда на перекрёстке'
ACTION_END = 'Мы доехали'
ACTION_STRAIGHT_CROSSROAD_TURN = 'Ехать до ближайшего перекрёстка и повернуть в направлении пункта назначения'
ACTION_CROSSROAD_TURN = 'Развернуться на ближайшем перекрёстке'

class Navigator(KnowledgeEngine):
    def declare_facts(self, facts):
        for fact in facts:
            self.declare(fact)

    @Rule(AND(Fact(navigator_direction = 'Прямо'),
              Fact(forbidden_sign = NO)))
    def forbidden_false(self):
        self.declare(Fact(final_action = ACTION_CONTINUE_DRIVING_STRAIGHT))

    @Rule(OR(Fact(navigator_direction = 'Направо'),
             Fact(navigator_direction = 'Налево')))
    def navigator_direction_right_left(self):
        self.declare(Fact(final_action = ACTION_CONTINUE_DRIVING_CROSSROAD))

    @Rule(Fact(navigator_direction = NO))
    def navigator_turn_around_false(self):
        self.declare(Fact(final_action = ACTION_END))

    @Rule(AND(Fact(navigator_direction = 'Назад'), 
              Fact(ban_on_turning = YES),
              Fact(one_way_traffic = YES)))
    def all_right_turn(self):
        self.declare(Fact(final_action = ACTION_STRAIGHT_CROSSROAD_TURN))

    @Rule(OR(
          AND(Fact(navigator_direction = 'Назад'),
              Fact(ban_on_turning = YES),
              Fact(one_way_traffic = NO)),
          AND(Fact(navigator_direction = 'Назад'),
              Fact(ban_on_turning = NO),
              Fact(stop_15_meters = YES))))
    def crossroad_turn(self):
        self.declare(Fact(final_action = ACTION_CROSSROAD_TURN))

    @Rule(OR(
          AND(Fact(navigator_direction = 'Прямо'), 
              Fact(forbidden_sign = YES)),
          AND(Fact(navigator_direction = 'Назад'),
              Fact(ban_on_turning = NO),
              Fact(stop_15_meters = NO))))
    def turn_true(self):
        self.declare(Fact(final_action = ACTION_TURN_AROUND))

    def factz(self,l):
        for x in l:
            self.declare(x)

    @Rule(Fact(final_action = MATCH.action), salience = 1)
    def what_to_do(self, action):
        print(action)

ModuleNotFoundError: No module named 'pyknow'

In [ ]:
test = Navigator()

In [ ]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Прямо'),
    Fact(forbidden_sign = 'Да')
])

test.run()

In [ ]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Назад'),
    Fact(ban_on_turning = YES),
    Fact(one_way_traffic = YES)
])

test.run()

In [ ]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Назад'),
    Fact(ban_on_turning = 'Да'),
    Fact(one_way_traffic = 'Нет')
])
test.run()

In [ ]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Прямо'),
    Fact(forbidden_sign = 'Нет')
])

test.run()

In [ ]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Направо')
])

test.run()

In [ ]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Направо')
])

test.run()

In [ ]:

test.reset()
test.factz([
    Fact(navigator_direction = 'Нет')
])

test.run()

In [ ]:
test.reset()
test.factz([
    Fact(navigator_direction = 'Назад'),
    Fact(ban_on_turning = 'Да'),
    Fact(one_way_traffic = 'Да')
])

test.run()